Imports

In [34]:
import pandas as pd
import numpy as np
import os
import unicodedata
import re

Normalize

In [35]:
# Function to normalize player names
def normalize_player_name(name):
    if not isinstance(name, str):
        return name
    
    # Replace common special characters
    replacements = {
        'á': 'a', 'à': 'a', 'â': 'a', 'ä': 'a', 'ã': 'a', 'å': 'a',
        'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e',
        'í': 'i', 'ì': 'i', 'î': 'i', 'ï': 'i',
        'ó': 'o', 'ò': 'o', 'ô': 'o', 'ö': 'o', 'õ': 'o', 'ø': 'o',
        'ú': 'u', 'ù': 'u', 'û': 'u', 'ü': 'u',
        'ý': 'y', 'ÿ': 'y',
        'ñ': 'n', 'ç': 'c', 'ß': 'ss', 'ð': 'd', 'þ': 'th',
        'Á': 'A', 'À': 'A', 'Â': 'A', 'Ä': 'A', 'Ã': 'A', 'Å': 'A',
        'É': 'E', 'È': 'E', 'Ê': 'E', 'Ë': 'E',
        'Í': 'I', 'Ì': 'I', 'Î': 'I', 'Ï': 'I',
        'Ó': 'O', 'Ò': 'O', 'Ô': 'O', 'Ö': 'O', 'Õ': 'O', 'Ø': 'O',
        'Ú': 'U', 'Ù': 'U', 'Û': 'U', 'Ü': 'U',
        'Ý': 'Y', 'Ÿ': 'Y',
        'Ñ': 'N', 'Ç': 'C', 'Ð': 'D', 'Þ': 'Th',
        'Ł': 'L', 'ł': 'l', 'Ś': 'S', 'ś': 's', 'Ź': 'Z', 'ź': 'z', 'Ż': 'Z', 'ż': 'z'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    # Normalize to ASCII form using unicodedata
    name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Remove any remaining non-alphanumeric characters except spaces
    name = re.sub(r'[^\w\s]', '', name)
    
    # Standardize whitespace
    name = ' '.join(name.split())
    
    return name

Read datas

In [36]:
# Read the combined data for 2021 and 2020
df_2021 = pd.read_excel('../Datas/combined_2021_data.xlsx')
df_2020 = pd.read_excel('../Datas/combined_2020_data.xlsx')
df_fbref = pd.read_excel('../Datas/FBREF_20_21.xlsx')

Merge 2020 and 2021 datas into a single data

In [37]:
# Find common players between the datasets
common_players = set(df_2021['Player']) & set(df_2020['Player'])
print(f"Number of common players between 2021 and 2020 datasets: {len(common_players)}")

# Filter datasets to only include common players
df_2021_filtered = df_2021[df_2021['Player'].isin(common_players)]
df_2020_filtered = df_2020[df_2020['Player'].isin(common_players)]

# Check for position changes (for informational purposes only)
position_changes = 0
for player in common_players:
    pos_2021 = df_2021_filtered.loc[df_2021_filtered['Player'] == player, 'Position'].values[0]
    pos_2020 = df_2020_filtered.loc[df_2020_filtered['Player'] == player, 'Position'].values[0]
    if pos_2021 != pos_2020:
        position_changes += 1
print(f"{position_changes} players have different positions between 2021 and 2020")

# Create a new DataFrame with only the columns we want in the exact order requested
transfermarkt_data = pd.DataFrame()
transfermarkt_data['Player'] = list(common_players)
transfermarkt_data = transfermarkt_data.merge(df_2020_filtered[['Player', 'Age']], on='Player', how='left')
transfermarkt_data = transfermarkt_data.merge(df_2020_filtered[['Player', 'Value (2020)']], on='Player', how='left')
transfermarkt_data = transfermarkt_data.merge(df_2021_filtered[['Player', 'Value (2021)']], on='Player', how='left')
transfermarkt_data = transfermarkt_data.merge(df_2021_filtered[['Player', 'League']], on='Player', how='left')
transfermarkt_data = transfermarkt_data.merge(df_2021_filtered[['Player', 'Position']], on='Player', how='left')

# Remove duplicates if any
transfermarkt_data = transfermarkt_data.drop_duplicates(subset=['Player'], keep='first')
print(f"After processing: {len(transfermarkt_data)} players in transfermarkt data")

# Save the transfermarkt dataset
transfermarkt_data.to_excel('../Datas/transfermarkt_scrape.xlsx', index=False)
print(f"Transfermarkt dataset created with {len(transfermarkt_data)} players and {len(transfermarkt_data.columns)} columns")

Number of common players between 2021 and 2020 datasets: 2336
5 players have different positions between 2021 and 2020
After processing: 2336 players in transfermarkt data
Transfermarkt dataset created with 2336 players and 6 columns


Merge transfermarkt_scrape.xlsx and FBREF_20_21.xlsx into a single file

In [38]:
# Load the datasets
transfermarkt_data = pd.read_excel('../Datas/transfermarkt_scrape.xlsx')
fbref_data = pd.read_excel('../Datas/FBREF_20_21.xlsx')

# Normalize player names in all datasets
print("Normalizing player names to handle special characters...")
transfermarkt_data['Player'] = transfermarkt_data['Player'].apply(normalize_player_name)
fbref_data['Player'] = fbref_data['Player'].apply(normalize_player_name)


# Make sure Player column in fbref_data is consistently named
if 'player' in fbref_data.columns and 'Player' not in fbref_data.columns:
    fbref_data = fbref_data.rename(columns={'player': 'Player'})

# Find common players between the datasets
common_players = set(transfermarkt_data['Player']) & set(fbref_data['Player'])
print(f"Number of common players between transfermarkt and FBREF datasets: {len(common_players)}")

# Filter datasets to only include common players
transfermarkt_filtered = transfermarkt_data[transfermarkt_data['Player'].isin(common_players)]
fbref_filtered = fbref_data[fbref_data['Player'].isin(common_players)]

# Merge the datasets on Player column
# Using left join to keep the order of players from transfermarkt_data
final_dataset = pd.merge(transfermarkt_filtered, fbref_filtered, on='Player', how='inner')

# Check for duplicates
duplicate_players = final_dataset[final_dataset.duplicated(subset=['Player'], keep=False)]
if len(duplicate_players) > 0:
    print(f"Found {len(duplicate_players)} duplicate player entries in the final dataset.")
    print("Duplicate players:")
    print(duplicate_players['Player'].unique())
    
    # Remove duplicates if any exist
    final_dataset = final_dataset.drop_duplicates(subset=['Player'], keep='first')

print(f"Final dataset contains {len(final_dataset)} players and {len(final_dataset.columns)} columns")

# Save the final dataset
final_dataset.to_excel('../Datas/final_dataset.xlsx', index=False)
print("Final dataset saved as 'final_dataset.xlsx'")

Normalizing player names to handle special characters...
Number of common players between transfermarkt and FBREF datasets: 1768
Found 2 duplicate player entries in the final dataset.
Duplicate players:
['Hernani']
Final dataset contains 1768 players and 141 columns
Final dataset saved as 'final_dataset.xlsx'


Remove duplicates

In [39]:
# Check for duplicate players before removing
duplicate_players = final_dataset[final_dataset.duplicated(subset=['Player'], keep=False)]
if len(duplicate_players) > 0:
    print(f"Found {len(duplicate_players)} duplicate player entries.")
    print("Duplicate players:")
    print(duplicate_players['Player'].unique())

# Remove duplicate rows based on 'Player' column, keeping the first occurrence
final_dataset = final_dataset.drop_duplicates(subset=['Player'], keep='first')
print(f"After removing duplicates: {len(final_dataset)} unique players remain")

After removing duplicates: 1768 unique players remain


Remove Goalkeepers because the stats do not suit goalkeepers


In [40]:
# Remove players those have the Position attribute set to Goalkeeper
final_dataset = final_dataset[final_dataset['Position'] != 'Goalkeeper']
print(f"After removing Goalkeepers: {len(final_dataset)} players remain")

After removing Goalkeepers: 1614 players remain


Group players into 3 categories: Defenders, Midfielders, Attackers

In [41]:
# Group Players into positions
Defenders = ['Left-Back', 'Right-Back', 'Centre-Back']
Midfielders = ['Defensive Midfield', 'Central Midfield', 'Attacking Midfield','Right Midfield', 'Left Midfield']
Forwards = ['Left Winger', 'Right Winger', 'Centre-Forward', 'Second Striker']

# Create a new column for Player Type
final_dataset['Category'] = 'Unknown'
final_dataset.loc[final_dataset['Position'].isin(Defenders), 'Category'] = 'Defender'
final_dataset.loc[final_dataset['Position'].isin(Midfielders), 'Category'] = 'Midfielder'
final_dataset.loc[final_dataset['Position'].isin(Forwards), 'Category'] = 'Forward'

# Get all column names
all_columns = final_dataset.columns.tolist()

# Remove 'Value Difference' from its current position
all_columns.remove('Category')

# Determine the first three columns (keep them in their original order)
first_three_columns = all_columns[:6]

# Insert 'Value Difference' as the 4th column
new_column_order = first_three_columns + ['Category'] + all_columns[6:]

# Reindex the DataFrame with the new column order
final_dataset = final_dataset[new_column_order]

Add value difference

In [42]:
# Add a new column for the difference in value between 2020 and 2021 that is Value (2021) - Value (2020) 
final_dataset['Value Difference'] = final_dataset['Value (2021)'] - final_dataset['Value (2020)']
# Get all column names
all_columns = final_dataset.columns.tolist()

# Remove 'Value Difference' from its current position
all_columns.remove('Value Difference')

# Determine the first three columns (keep them in their original order)
first_three_columns = all_columns[:7]

# Insert 'Value Difference' as the 4th column
new_column_order = first_three_columns + ['Value Difference'] + all_columns[7:]

# Reindex the DataFrame with the new column order
final_dataset = final_dataset[new_column_order]

Add value difference for players who have nonzero values at 2020 and 2021 as percentage 


In [43]:
# Remove players with 0 value in Value (2020)
final_dataset = final_dataset[final_dataset['Value (2020)'] != 0]
# Remove players with 0 value in Value (2021)
final_dataset = final_dataset[final_dataset['Value (2021)'] != 0]

# Add a new column for the difference in value between 2020 and 2021 that is Value (2021) - Value (2020) 
final_dataset['Value Difference (%)'] = (final_dataset['Value (2021)'] - final_dataset['Value (2020)'])/final_dataset['Value (2020)'] * 100
# Get all column names
all_columns = final_dataset.columns.tolist()

# Remove 'Value Difference' from its current position
all_columns.remove('Value Difference (%)')

# Determine the first three columns (keep them in their original order)
first_three_columns = all_columns[:8]

# Insert 'Value Difference' as the 4th column
new_column_order = first_three_columns + ['Value Difference (%)'] + all_columns[8:]

# Reindex the DataFrame with the new column order
final_dataset = final_dataset[new_column_order]


Save

In [44]:
final_dataset.to_excel('../Datas/final_dataset.xlsx', index=False)



Print

In [45]:
print(f"Combined dataset created with {len(final_dataset)} players and {len(final_dataset.columns)} columns")

Combined dataset created with 1608 players and 144 columns
